In [ ]:
# Step 1: Create folders to store PNG files created from DICOM

In [ ]:
### Load Essential Packages ###
import os

### Create New Directories Based on Directory Template ###
for root, dirs, files in os.walk('/media/bitlockermount/MESArthritis/'):
    path = root.split(os.sep)
    path2 = path[-1]
    path3 = ''.join(path2)
    os.makedirs('/home/john/Documents/MESArthritis/Ax_Cor_Sag/' + path3, exist_ok=True)

In [ ]:
# Step 2: Select the correct DICOM/PNG series for interpretation

In [ ]:
#####WINDOWING#####

###head and neck
#brain W:80 L:40
#subdural W:130-300 L:50-100
#stroke W:8 L:32 or W:40 L:40 3
#temporal bones W:2800 L:600 or W:4000 L:700
#soft tissues: W:350–400 L:20–60 4

###chest
#lungs W:1500 L:-600
#mediastinum W:350 L:50

###abdomen
#soft tissues W:400 L:50
#liver W:150 L:30

###spine
#soft tissues W:250 L:50
#bone W:1800 L:400


#####Path Sorting####
###axial = 10:-4
###coronal = 12:-4
###sagittal = 13:-4

### Load Essential Packages ###
import pydicom
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import sys
import glob
import csv
import cv2
import os
from os import listdir
from os.path import isfile, join

### Set Directory of DICOMs ###
folder = 'path/to/folder/with/DICOM'
sub_folders = [name for name in os.listdir(folder) if os.path.isdir(os.path.join(folder, name))]
sortedfolder = sorted(sub_folders)

### Sort folders for Primary Axial Sequences ###
for q in sortedfolder:
    files = []
    
    mypath = 'path/to/folder/with/DICOM/' + q + '/'
    onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
    selectedfiles = [x[0:7] for x in onlyfiles]

    def most_frequent(List):
        return max(set(List), key = List.count)
    common = most_frequent(selectedfiles)

### Automatically Create PNG from DICOM for All Patients in Directory ###
    for filename in sorted(glob.glob('path/to/folder/with/DICOM/' + q + '/' + common + '*.dcm')):
        files.append(pydicom.dcmread(filename))

    print("file count: {}".format(len(files)))

    # skip files with no SliceLocation (eg scout views)
    slices = []
    skipcount = 0
    for f in files:
        if hasattr(f, 'SliceLocation'):
            slices.append(f)
        else:
            skipcount = skipcount + 1

    print("skipped, no SliceLocation: {}".format(skipcount))

### Ensure Correct Order of Files ###
    slices = sorted(slices, key=lambda s: s.SliceLocation)

### Capture Pixel Spacing and Slice Thickness ###
    ps1 = []
    ps2 = []
    ss = []
    man = []
    model = []
    imagetype = []

### Set Mediastinal Windowing ###
    ww=350
    wl=50
    U = 255
    W = U / ww
    b = (-U/ww) * (wl-ww/2)    
    
    try:
        
### Fill Axial Array with Images in the File ###
        m = 0
        for i, s in enumerate(slices):
            img = s.pixel_array
            intercept = float(s.RescaleIntercept)
            slope = float(s.RescaleSlope)
            img2d = slope * img + intercept
            axial = img2d
            axial = W * axial + b
            axial = np.where(axial > U, U, axial)
            axial = np.where(axial < 0, 0, axial)
            axial -= axial.mean()
            axial /= (axial.std() + 1e-10)
            axial -= axial.min()
            axial = (255 * axial/np.max(axial)).astype('uint8')
            axial2 = cv2.resize(axial, (512, 512), interpolation = cv2.INTER_NEAREST) 
            cv2.imwrite('path/to/saved/file/' + q + '/axial_' + common + '_' + str(m) + '.png', axial2)

            m += 1

        ps1, ps2 = slices[0].PixelSpacing
        ss = slices[0].SliceThickness
        man = slices[0].Manufacturer
        model = slices[0].ManufacturerModelName
        imagetype = slices[0].ImageType
        fields = [q, common, ps1, ps2, ss, man, model, imagetype]
        with open('path/to/saved/pixel/spacing/.csv', 'a') as f:
            writer = csv.writer(f)
            writer.writerow(fields)
        
            m += 1
    except TypeError:
        pass

In [ ]:
# Step 3: Select the frame directly above the aortic arch

In [ ]:
### Load Essential Packages ###
import numpy as np
import pandas as pd
import os
import cv2
from imutils import paths
import argparse
import glob
from glob import iglob

n_white_pix = []
names = []
file_count = []
index = 0

### Select Frame Directly Above the Aorta ###
for filename in sorted(glob.glob('path/to/PNG/files/**/axial_*.png'), key=lambda name: (name[45:56], int(os.path.basename(name)[14:-4]))):
    image = cv2.imread(filename)
    names.append(filename)
    pathlen, filenamed = os.path.split(filename)
    file_count = len(glob.glob1(pathlen,"axial_*.png"))
    image[image <= 80] = 0
    image[image >= 140] = 0
    cropped = image[165:360, 180:332]
    n_white_pix = np.append(n_white_pix, np.sum(cropped > 0))
    index += 1

### Select the Array of Images to Search and Apply Pixel Intensities to Stratify ###
    if index == file_count:
        lower = int(file_count*0.6)
        upper = int(file_count*0.85)
        minimum = (np.argmin(n_white_pix[lower:upper]) + lower)
        midline = cv2.imread(names[minimum])
        path, filenames = os.path.split(names[minimum])
        path2 = os.path.basename(path)
        cv2.imwrite('path/to/saved/file/' + path2 + filenames, midline)
        
        n_white_pix = []
        names = []
        file_count = []
        index = 0

    else:
        continue

In [ ]:
# Step 4: Custom semantic segmentation using Pixellib and Mask R-CNN for all remaining participants

In [ ]:
### Load Essential Packages ###
import pixellib
from pixellib.custom_train import instance_custom_training

### Train the MASK R-CNN Model from the Groundtruth Images ###
train_maskrcnn = instance_custom_training()
train_maskrcnn.modelConfig(network_backbone = "resnet50", num_classes= 3, batch_size = 12)
train_maskrcnn.load_pretrained_model("path/to//mask_rcnn_coco.h5")
train_maskrcnn.load_dataset("path/to/manual/segementation")
train_maskrcnn.train_model(num_epochs = 4, augmentation=False,  path_trained_models = "path/to/saved/models/")

In [ ]:
### Load Essential Packages ###
import pixellib
from pixellib.custom_train import instance_custom_training

### Evaluate performance of the MASK R-CNN Model using the Test/Holdout Group ###
train_maskrcnn = instance_custom_training()
train_maskrcnn.modelConfig(network_backbone = "resnet50", num_classes= 3, batch_size = 1)
train_maskrcnn.load_pretrained_model("path/to/mask_rcnn_coco.h5")
train_maskrcnn.load_dataset("path/to/manual/segementation")
train_maskrcnn.evaluate_model("path/to/trained/model/.h5")

In [ ]:
### Load Essential Packages ###
from pixellib.instance import custom_segmentation

### Custom Segmentation of Left PMAT, Right PMAT, and SAT ###
segment_image = custom_segmentation()
segment_image.inferConfig(num_classes = 3, class_names= ["BG", "PMAT_L", "PMAT_R", "SAT"], detection_threshold = 0.9, network_backbone = "resnet50")
segment_image.load_model("path/to/trained/model/.h5")

In [ ]:
### Load Essential Packages ###
import numpy as np
from PIL import Image
import cv2
import os

### Set Directories ###
inputdir = 'path/to/Infer/'
outdir = 'path/to/Output/'
outdir1 = 'path/to/PMR_Mask_'
outdir2 = 'path/to/PMR_'
outdir3 = 'path/to/PML_Mask_'
outdir4 = 'path/to/PML_'
outdir5 = 'path/to/SAT_Mask_'
outdir6 = 'path/to/SAT_'

test_list = [ f for f in  os.listdir(inputdir)]

### Inference of Left PMAT, Right PMAT, and SAT Images Based on Mask R-CNN Model ###
for f in sorted (test_list):
    if f.endswith('.png'):
        segmask, output = segment_image.segmentImage(inputdir + f, extract_segmented_objects = False, mask_points_values = True, show_bboxes=False)
        class_list = segmask['class_ids'].tolist()
        class_str = ''.join(str(x) for x in class_list)
        try:
            if class_str == '321':
                c,b,a = [ [individualArray] for individualArray in segmask['masks'] ]
            elif class_str == '312':
                c,a,b = [ [individualArray] for individualArray in segmask['masks'] ]
            elif class_str == '231':
                b,c,a = [ [individualArray] for individualArray in segmask['masks'] ]
            elif class_str == '213':
                b,a,c = [ [individualArray] for individualArray in segmask['masks'] ]
            elif class_str == '132':
                a,c,b = [ [individualArray] for individualArray in segmask['masks'] ]
            else:
                a,b,c = [ [individualArray] for individualArray in segmask['masks'] ]
        except:
            pass

        cv2.imwrite(outdir + f, output)
        a1 = np.array(a)
        b1 = np.array(b)
        c1 = np.array(c)
        for i in a1.tolist():
            mask1 = np.asarray(Image.open(inputdir + f))
            mask11 =  mask1[:,:,:3]
            img1 = cv2.fillPoly(mask1, np.array(i), color=(0, 0, 0))
            maskB = Image.fromarray(mask11)
        for i in a1.tolist():
            mask2 = np.asarray(Image.open(inputdir + f))
            mask22 =  mask2[:,:,:3]
            img2 = cv2.fillPoly(mask2, np.array(i), color=(255, 255, 255))
            maskW = Image.fromarray(mask22)
            ROI_PMR_Mask = np.subtract(maskW, maskB)
            ROI_PMR = np.subtract(np.asarray(Image.open(inputdir + f))[:,:,:3], maskB)
            cv2.imwrite(outdir1 + f, ROI_PMR_Mask)
            cv2.imwrite(outdir2 + f, ROI_PMR)
        for i in b1.tolist():
            mask3 = np.asarray(Image.open(inputdir + f))
            mask33 =  mask3[:,:,:3]
            img3 = cv2.fillPoly(mask3, np.array(i), color=(0, 0, 0))
            maskB2 = Image.fromarray(mask33)
        for i in b1.tolist():
            mask4 = np.asarray(Image.open(inputdir + f))
            mask44 =  mask4[:,:,:3]
            img4 = cv2.fillPoly(mask4, np.array(i), color=(255, 255, 255))
            maskW2 = Image.fromarray(mask44)
            ROI_SAT_Mask = np.subtract(maskW2, maskB2)
            ROI_SAT = np.subtract(np.asarray(Image.open(inputdir + f))[:,:,:3], maskB2)
            cv2.imwrite(outdir3 + f, ROI_SAT_Mask)
            cv2.imwrite(outdir4 + f, ROI_SAT)
        for i in c1.tolist():
            mask5 = np.asarray(Image.open(inputdir + f))
            mask55 =  mask5[:,:,:3]
            img5 = cv2.fillPoly(mask5, np.array(i), color=(0, 0, 0))
            maskB3 = Image.fromarray(mask55)
        for i in c1.tolist():
            mask6 = np.asarray(Image.open(inputdir + f))
            mask66 =  mask6[:,:,:3]
            img6 = cv2.fillPoly(mask6, np.array(i), color=(255, 255, 255))
            maskW3 = Image.fromarray(mask66)
            ROI_PML_Mask = np.subtract(maskW3, maskB3)
            ROI_PML = np.subtract(np.asarray(Image.open(inputdir + f))[:,:,:3], maskB3)
            cv2.imwrite(outdir5 + f, ROI_PML_Mask)
            cv2.imwrite(outdir6 + f, ROI_PML)

In [ ]:
# Step 5: Export the manual segementation binary mask layers for PMAT and SAT

In [ ]:
### Load Essential Packages ###
import glob
import json
import os
import os.path as osp

import numpy as np
import PIL.Image
import cv2
import labelme

### Export Binary Mask Layer for Left PMAT, Right PMAT, and SAT ###
DATA_DIR = 'path/to/Development/train'
OUT_DIR = 'path/to/Ground_Truth/train'
class_names = []
class_name_to_id = {}
for i, line in enumerate(open(DATA_DIR + '/labels.txt').readlines()):
    class_id = i
    class_name = line.strip()
    class_name_to_id[class_name] = class_id
#    if class_id == -1:
#        class_name == '__ignore__'
#        continue
#    elif class_id == 0:
#        class_name == '_background_'
    class_names.append(class_name)
class_names = tuple(class_names)
print('class_names:', class_names)
out_class_names_file = osp.join(DATA_DIR, 'class_names.txt')
with open(out_class_names_file, 'w') as f:
    f.writelines('\n'.join(class_names))
print('Saved class_names:', out_class_names_file)

if osp.exists(OUT_DIR):
    print('Output directory already exists:', OUT_DIR)
    quit(1)
os.makedirs(OUT_DIR, exist_ok=True)
for label_file in sorted(glob.glob(osp.join(DATA_DIR, '*.json'))):
    with open(label_file) as f:
        base = osp.splitext(osp.basename(label_file))[0]
        data = json.load(f)
        img_file = osp.join(osp.dirname(label_file), data['imagePath'])
        img = np.asarray(PIL.Image.open(img_file))
        lbl, _ = labelme.utils.shapes_to_label(
            img_shape=img.shape,
            shapes=data['shapes'],
            label_name_to_value=class_name_to_id,
        )
        instance1 = np.copy(lbl)
        instance2 = np.copy(lbl)
        instance3 = np.copy(lbl)
        pos_1 = np.where(lbl==0)
        pos_2 = np.where(lbl==1)
        pos_3 = np.where(lbl==2)
        instance1[pos_1] = 0
        instance2[pos_2] = 0
        instance3[pos_3] = 0

### Create the Correct Layers ###
        instance1 = instance1*255
        instance2 = instance2*255
        instance3 = instance3*255
        instance4 = np.subtract(instance1, instance2)
        instance5 = np.subtract(instance1, instance3)
        instance6 = np.subtract(instance3, instance4)
        os.makedirs(osp.join(OUT_DIR, base), exist_ok=True)
        os.makedirs(osp.join(OUT_DIR, base,'images'), exist_ok=True)
        PIL.Image.fromarray(img).save(osp.join(OUT_DIR, base, 'images', base + '.png'))
        os.makedirs(osp.join(OUT_DIR, base, 'masks'), exist_ok=True)
        cv2.imwrite(osp.join(OUT_DIR, base, 'masks', base + '_ALL' + '.png'), instance1)
        cv2.imwrite(osp.join(OUT_DIR, base, 'masks', base + '_PML' + '.png'), instance4)
        cv2.imwrite(osp.join(OUT_DIR, base, 'masks', base + '_PMR' + '.png'), instance5)
        cv2.imwrite(osp.join(OUT_DIR, base, 'masks', base + '_SAT' + '.png'), instance6)

In [ ]:
# Step 6: Determine the total number of pixels within the ROI

In [ ]:
### Load Essential Packages ###
import cv2
import os
import numpy as np
import glob
from glob import iglob
import csv

### Count White Pixels in Image ###
for filename in sorted(glob.glob('path/to/ROI/**/*.png')):
    img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
    n_white_pix = np.sum(img == 255)
    average = img[img!=0].mean()
    path, filenames = os.path.split(filename)
    path2 = os.path.basename(path)
    if "Mask" in filenames:
        name_type = "Mask"
    else:
        name_type = "Image"
    if "PML_Mask_" in filenames:
        name_region = "Pectoralis Muscle, Left"
    elif "PMR_Mask_" in filenames:
        name_region = "Pectoralis Muscle, Right"
    elif "PML-PATL_Mask_" in filenames:
        name_region = "Pectoralis Muscle, Left Subtracted from Intermuscular Adipose Tissue, Left"
    elif "PMR-PATR_Mask_" in filenames:
        name_region = "Pectoralis Muscle, Right Subtracted from Intermuscular Adipose Tissue, Right"    
    elif "PML-PATL_MESA" in filenames:
        name_region = "Pectoralis Muscle, Left Subtracted from Intermuscular Adipose Tissue, Left"
    elif "PMR-PATR_MESA" in filenames:
        name_region = "Pectoralis Muscle, Right Subtracted from Intermuscular Adipose Tissue, Right"    
    elif "EMLL_Mask_" in filenames:
        name_region = "Extramyocellular Lipid, Left"
    elif "EMLR_Mask_" in filenames:
        name_region = "Extramyocellular Lipid, Right"
    elif "SAT_Mask_" in filenames:
        name_region = "Subcutaneous Adipose Tissue"
    elif "PATL_Mask_" in filenames:
        name_region = "Intermuscular Adipose Tissue, Left"
    elif "PATR_Mask_" in filenames:
        name_region = "Intermuscular Adipose Tissue, Right"     
    fields=[path2, name_type, name_region, filenames, n_white_pix, average]
    with open('path/to/Measurements_Mask.csv', 'a') as f:
        writer = csv.writer(f)
        writer.writerow(fields)

In [ ]:
# Step 7: Create a combined ROI for Dice and IoU Scoring

In [ ]:
### Load Essential Packages ###
from PIL import Image
import numpy as np
import cv2
import os
import glob
from glob import iglob

### Combine SAT, PMR, and PML ###
for file1 in sorted(glob.glob('path/to/**/**/*SAT_Mask_MESA*.png')):
    for file2 in sorted(glob.glob('path/to/**/**/*PMR_Mask_MESA*.png')):
        if file1[54:65] == file2[54:65]:
            for file3 in sorted(glob.glob('path/to/**/**/*PML_Mask_MESA*.png')):
                if file1[54:65] == file3[54:65]:
                    path, filenames = os.path.split(file1)
                    path2 = os.path.basename(path)
                    One = Image.open(file1)
                    Two = Image.open(file2)
                    Three = Image.open(file3)
                    One1 = np.array(One)
                    Two2 = np.array(Two)
                    Three3 = np.array(Three)
                    addition = One1 + Two2 + Three3
    cv2.imwrite(path + '/ALL_' + filenames[9:], addition)

In [ ]:
# Step 8: Calculate Dice score and IoU in the training and testing sets

In [ ]:
from PIL import Image
import SimpleITK as sitk
import cv2
import os
import numpy as np
import glob
from glob import iglob
import csv

for file1 in sorted(glob.glob('path/to/**/**/*_SAT.png')):
    for file2 in sorted(glob.glob('path/to/**/**/*SAT_Mask_MESA*.png')):
        if file1[54:65] == file2[54:65]:
            path, filenames = os.path.split(file1)
            path2 = os.path.basename(path)
            img = Image.open(file1)
            img2 = Image.open(file2).convert("L")
            img = np.array(img)
            img2 = np.array(img2)
            intersection = np.logical_and(img, img2)
            union = np.logical_or(img, img2)
            iou_score = np.sum(intersection) / np.sum(union)
            dice_score = (2*np.sum(intersection))/(np.sum(union)+np.sum(intersection))
            mean_pixel_accuracy = 1.0 * np.sum(img) / (np.spacing(1) + np.sum(img2))
            fields=[filenames[0:11], "SAT", iou_score, dice_score, mean_pixel_accuracy]
    with open('path/to/Metrics.csv', 'a') as f:
        writer = csv.writer(f)
        writer.writerow(fields)